# Change Idea 1: check for matching multipoles

Instead of allowing array size mismatch crashes to occur in the somewhat cryptic `run_taylens()`, I've added a check in the `__init__()` method, which should help the end-user:

```python
        # Ensure multipoles are the same length if delensing
        if self.apply_delens:
            ell_delens  = self.delensing_ells[0]
            ell_spectra = self.cmb_spectra[0]
            if not np.array_equal(ell_delens, ell_spectra):
                # Multipoles do not match
                raise ValueError("Multipoles do not match for cmb_spectra and "\
                                 "delensing ells. Check input files.")
```

This may be premature optimization. It's in lines 49-57 in the next cell.

In [ ]:
class CMBLensed(CMBMap):
    # inherit from CMBMap so we get the `get_emission` method
    def __init__(
        self,
        nside,
        cmb_spectra,
        max_nside=None,
        cmb_seed=None,
        apply_delens=False,
        delensing_ells=None,
        map_dist=None,
    ):
        """Lensed CMB

        Takes an input unlensed CMB and lensing spectrum from CAMB and uses
        Taylens to apply lensing, it optionally simulates delensing by
        suppressing the lensing power at specific scales with the user
        provided `delensing_ells`.

        Parameters
        ----------

        cmb_spectra : path
            Input text file from CAMB, spectra unlensed
        cmb_seed : int
            Numpy random seed for synfast, set to None for a random seed
        apply_delens : bool
            If true, simulate delensing with taylens
        delensing_ells : path
            Space delimited file with ells in the first columns and suppression
            factor (1 for no suppression) in the second column
        """
        try:
            super().__init__(nside=nside, max_nside=max_nside, map_dist=map_dist)
        except ValueError:
            pass  # suppress exception about not providing any input map
        self.cmb_spectra = self.read_txt(cmb_spectra, unpack=True)
        self.cmb_seed = cmb_seed
        self.apply_delens = apply_delens
        self.delensing_ells = (
            None if delensing_ells is None else self.read_txt(delensing_ells)
        )

        # Remove monopole and dipole, if present
        if self.cmb_spectra[0][0] == 0:
            self.cmb_spectra = self.cmb_spectra[:, 2:]
        if self.apply_delens and self.delensing_ells[0][0] == 0:
            self.delensing_ells = self.delensing_ells[:, 2:]
        # Ensure multipoles are the same length if delensing
        if self.apply_delens:
            ell_delens  = self.delensing_ells[0]
            ell_spectra = self.cmb_spectra[0]
            if not np.array_equal(ell_delens, ell_spectra):
                # Multipoles do not match
                raise ValueError("Multipoles do not match for cmb_spectra and "\
                                 "delensing ells. Check input files.")

        self.map = u.Quantity(self.run_taylens(), unit=u.uK_CMB, copy=False)


This code could be more modular, making it more flexible.

In [ ]:
import healpy as hp
import numpy as np
import matplotlib.pyplot as plt
import pysm3.units as u
from pysm3.models.cmb import CMBMap
from pysm3.models.cmb import CMBLensed as CMBLensedOriginal

# Change 2: Separate loading spectra

Making the code more modular allows end users more flexibility.

In [ ]:
class CMBLensed(CMBMap):
    # inherit from CMBMap so we get the `get_emission` method
    def __init__(
        self,
        nside,
        cmb_spectra,
        max_nside=None,
        cmb_seed=None,
        apply_delens=False,
        delensing_ells=None,
        map_dist=None,
    ):
        """Lensed CMB

        Takes an input unlensed CMB and lensing spectrum from CAMB and uses
        Taylens to apply lensing, it optionally simulates delensing by
        suppressing the lensing power at specific scales with the user
        provided `delensing_ells`.

        Parameters
        ----------

        cmb_spectra : path
            Input text file from CAMB, spectra unlensed
        cmb_seed : int
            Numpy random seed for synfast, set to None for a random seed
        apply_delens : bool
            If true, simulate delensing with taylens
        delensing_ells : path
            Space delimited file with ells in the first columns and suppression
            factor (1 for no suppression) in the second column
        """
        try:
            super().__init__(nside=nside, max_nside=max_nside, map_dist=map_dist)
        except ValueError:
            pass  # suppress exception about not providing any input map
        self.cmb_seed = cmb_seed
        self.apply_delens = apply_delens

        self.cmb_spectra, self.delensing_ells = self.load_spectra(cmb_spectra, 
                                                                  apply_delens, 
                                                                  delensing_ells)

        self.map = u.Quantity(self.run_taylens(), unit=u.uK_CMB, copy=False)

    @staticmethod
    def load_spectra(cmb_spectra, apply_delens, delensing_ells):
        # Read in the spectra
        cmb_spectra = CMBLensed.read_txt(cmb_spectra, unpack=True)
        if apply_delens:
            delensing_ells = CMBLensed.read_txt(delensing_ells)

        # Remove monopole and dipole, if present
        if cmb_spectra[0][0] == 0:
            cmb_spectra = cmb_spectra[:, 2:]
        if apply_delens and delensing_ells[0][0] == 0:
            delensing_ells = delensing_ells[:, 2:]
        # Ensure multipoles are the same length if delensing
        if apply_delens:
            ell_delens  = delensing_ells[0]
            ell_spectra = delensing_ells[0]
            if not np.array_equal(ell_delens, ell_spectra):
                # Multipoles do not match
                raise ValueError("Multipoles do not match for cmb_spectra and "\
                                 "delensing ells. Check input files.")
        return cmb_spectra, delensing_ells

## Break up run_taylens()

### Original

In [ ]:
def run_taylens(self):
    """Returns CMB (T, Q, U) maps as a function of observing frequency, nu.

    This code is extracted from the taylens code (reference).

    :return: function -- CMB maps.
    """
    synlmax = 8 * self.nside  # this used to be user-defined.
    data = self.cmb_spectra
    lmax_cl = len(data[0]) + 1
    ell = np.arange(int(lmax_cl + 1))
    synlmax = min(synlmax, ell[-1])

    # Reading input spectra in CAMB format. CAMB outputs l(l+1)/2pi hence the corrections.
    cl_tebp_arr = np.zeros([10, lmax_cl + 1])
    cl_tebp_arr[0, 2:] = 2 * np.pi * data[1] / (ell[2:] * (ell[2:] + 1))  # TT
    cl_tebp_arr[1, 2:] = 2 * np.pi * data[2] / (ell[2:] * (ell[2:] + 1))  # EE
    cl_tebp_arr[2, 2:] = 2 * np.pi * data[3] / (ell[2:] * (ell[2:] + 1))  # BB
    cl_tebp_arr[4, 2:] = 2 * np.pi * data[4] / (ell[2:] * (ell[2:] + 1))  # TE
    cl_tebp_arr[5, :] = np.zeros(lmax_cl + 1)  # EB
    cl_tebp_arr[7, :] = np.zeros(lmax_cl + 1)  # TB

    if self.apply_delens:
        cl_tebp_arr[3, 2:] = (
            2
            * np.pi
            * data[5]
            * self.delensing_ells[1]
            / (ell[2:] * (ell[2:] + 1)) ** 2
        )  # PP
        cl_tebp_arr[6, :] = np.zeros(lmax_cl + 1)  # BP
        cl_tebp_arr[8, 2:] = (
            2
            * np.pi
            * data[7]
            * np.sqrt(self.delensing_ells[1])
            / (ell[2:] * (ell[2:] + 1)) ** 1.5
        )  # EP
        cl_tebp_arr[9, 2:] = (
            2
            * np.pi
            * data[6]
            * np.sqrt(self.delensing_ells[1])
            / (ell[2:] * (ell[2:] + 1)) ** 1.5
        )  # TP
    else:
        cl_tebp_arr[3, 2:] = (
            2 * np.pi * data[5] / (ell[2:] * (ell[2:] + 1)) ** 2
        )  # PP
        cl_tebp_arr[6, :] = np.zeros(lmax_cl + 1)  # BP
        cl_tebp_arr[8, 2:] = (
            2 * np.pi * data[7] / (ell[2:] * (ell[2:] + 1)) ** 1.5
        )  # EP
        cl_tebp_arr[9, 2:] = (
            2 * np.pi * data[6] / (ell[2:] * (ell[2:] + 1)) ** 1.5
        )  # TP

    # Coordinates of healpix pixel centers
    ipos = np.array(hp.pix2ang(self.nside, np.arange(hp.nside2npix(self.nside))))

    # Simulate a CMB and lensing field
    cmb, aphi = simulate_tebp_correlated(
        cl_tebp_arr, self.nside, synlmax, self.cmb_seed
    )

    if cmb.ndim == 1:
        cmb = np.reshape(cmb, [1, cmb.size])

    # Compute the offset positions
    phi, phi_dtheta, phi_dphi = hp.alm2map_der1(aphi, self.nside, lmax=synlmax)

    del aphi

    opos, rot = offset_pos(
        ipos, phi_dtheta, phi_dphi, pol=True, geodesic=False
    )  # geodesic used to be used defined.
    del phi, phi_dtheta, phi_dphi

    # Interpolate maps one at a time
    maps = []
    for comp in cmb:
        for m in taylor_interpol_iter(
            comp, opos, 3, verbose=False, lmax=None
        ):  # lmax here needs to be fixed. order of taylor expansion is fixed to 3.
            pass
        maps.append(m)
    del opos, cmb
    return np.array(apply_rotation(maps, rot))


### Changed:

Simple separation:

In [ ]:
def convert_spectra(self, data, lmax_cl):
    # Reading input spectra in CAMB format. CAMB outputs l(l+1)/2pi hence the corrections.
    cl_tebp_arr = np.zeros([10, lmax_cl + 1])
    cl_tebp_arr[0, 2:] = 2 * np.pi * data[1] / (ell[2:] * (ell[2:] + 1))  # TT
    cl_tebp_arr[1, 2:] = 2 * np.pi * data[2] / (ell[2:] * (ell[2:] + 1))  # EE
    cl_tebp_arr[2, 2:] = 2 * np.pi * data[3] / (ell[2:] * (ell[2:] + 1))  # BB
    cl_tebp_arr[4, 2:] = 2 * np.pi * data[4] / (ell[2:] * (ell[2:] + 1))  # TE
    cl_tebp_arr[5, :] = np.zeros(lmax_cl + 1)  # EB
    cl_tebp_arr[6, :] = np.zeros(lmax_cl + 1)  # BP
    cl_tebp_arr[7, :] = np.zeros(lmax_cl + 1)  # TB

    if self.apply_delens:
        cl_tebp_arr[3, 2:] = 2 * np.pi * data[5] * self.delensing_ells[1] /\
            (ell[2:] * (ell[2:] + 1)) ** 2  # PP
        cl_tebp_arr[8, 2:] = 2 * np.pi * data[7] * np.sqrt(self.delensing_ells[1]) /\
            (ell[2:] * (ell[2:] + 1)) ** 1.5  # EP
        cl_tebp_arr[9, 2:] = 2 * np.pi * data[6] * np.sqrt(self.delensing_ells[1]) /\
            (ell[2:] * (ell[2:] + 1)) ** 1.5  # TP
    else:
        cl_tebp_arr[3, 2:] = 2 * np.pi * data[5] /\
              (ell[2:] * (ell[2:] + 1)) ** 2  # PP
        cl_tebp_arr[8, 2:] = 2 * np.pi * data[7] /\
              (ell[2:] * (ell[2:] + 1)) ** 1.5  # EP
        cl_tebp_arr[9, 2:] = 2 * np.pi * data[6] /\
              (ell[2:] * (ell[2:] + 1)) ** 1.5  # TP

def run_taylens(self):
    """Returns CMB (T, Q, U) maps as a function of observing frequency, nu.

    This code is extracted from the taylens code (reference).

    :return: function -- CMB maps.
    """
    synlmax = 8 * self.nside  # this used to be user-defined.
    data = self.cmb_spectra
    lmax_cl = len(data[0]) + 1
    ell = np.arange(int(lmax_cl + 1))
    synlmax = min(synlmax, ell[-1])

    cl_tebp_arr = convert_spectra(data)

    # Coordinates of healpix pixel centers
    ipos = np.array(hp.pix2ang(self.nside, np.arange(hp.nside2npix(self.nside))))

    # Simulate a CMB and lensing field
    cmb, aphi = simulate_tebp_correlated(
        cl_tebp_arr, self.nside, synlmax, self.cmb_seed
    )

    if cmb.ndim == 1:
        cmb = np.reshape(cmb, [1, cmb.size])

    # Compute the offset positions
    phi, phi_dtheta, phi_dphi = hp.alm2map_der1(aphi, self.nside, lmax=synlmax)

    del aphi

    opos, rot = offset_pos(
        ipos, phi_dtheta, phi_dphi, pol=True, geodesic=False
    )  # geodesic used to be used defined.
    del phi, phi_dtheta, phi_dphi

    # Interpolate maps one at a time
    maps = []
    for comp in cmb:
        for m in taylor_interpol_iter(
            comp, opos, 3, verbose=False, lmax=None
        ):  # lmax here needs to be fixed. order of taylor expansion is fixed to 3.
            pass
        maps.append(m)
    del opos, cmb
    return np.array(apply_rotation(maps, rot))


Better separation, but rearrangement causes numerical errors:

In [ ]:
def build_tebp_dl2cl_scale(lmax):
    """
    Builds scale factors for converting between bandpower and angular
    power spectrum.
    """
    ell = np.arange(2, lmax + 1)

    scale = np.zeros([10, ell.size]) 
    scale[0] = 2 * np.pi / (ell * (ell + 1))         # TT
    scale[1] = 2 * np.pi / (ell * (ell + 1))         # EE
    scale[2] = 2 * np.pi / (ell * (ell + 1))         # BB
    scale[3] = 2 * np.pi / (ell * (ell + 1)) ** 2    # PP
    scale[4] = 2 * np.pi / (ell * (ell + 1))         # TE
    scale[5] = 2 * np.pi / (ell * (ell + 1))         # EB
    scale[6] = 2 * np.pi / (ell * (ell + 1))         # BP
    scale[7] = 2 * np.pi / (ell * (ell + 1))         # TB
    scale[8] = 2 * np.pi / (ell * (ell + 1)) ** 1.5  # EP
    scale[9] = 2 * np.pi / (ell * (ell + 1)) ** 1.5  # TP

    return scale


def camb2tebp(camb_dl, delens_ells=None):
    """
    Converts CAMB output to the TEBP format expected by the Taylens code

    Assumptions:
    - Multipoles are in order, non-repeating, continuous
    - CAMB output is L, TT, EE, BB, TE, PP, TP, EP
    - Multipoles start at 2
    """
    lmax = int(camb_dl[0][-1])  # Last value in the first column (L)

    # Dipoles start at 2; need lmax-2 values -> shape of ells would be lmax-1
    _cl_tebp_arr = np.zeros([10, lmax - 1])
    _cl_tebp_arr[0] = camb_dl[1]  # TT
    _cl_tebp_arr[1] = camb_dl[2]  # EE
    _cl_tebp_arr[2] = camb_dl[3]  # BB
    _cl_tebp_arr[3] = camb_dl[5]  # PP
    _cl_tebp_arr[4] = camb_dl[4]  # TE
    _cl_tebp_arr[5] = 0           # EB
    _cl_tebp_arr[6] = 0           # BP
    _cl_tebp_arr[7] = 0           # TB
    _cl_tebp_arr[8] = camb_dl[7]  # EP
    _cl_tebp_arr[9] = camb_dl[6]  # TP

    # Convert band power to power spectra
    scale = build_tebp_dl2cl_scale(lmax)
    _cl_tebp_arr *= scale

    if delens_ells is not None:
        # Multiply the lensing power spectrum by the delensing factor
        _cl_tebp_arr[3] *= delens_ells[1]
        _cl_tebp_arr[8] *= np.sqrt(delens_ells[1])
        _cl_tebp_arr[9] *= np.sqrt(delens_ells[1])

    # Create final array with zeros for monopole and dipole
    cl_tebp_arr = np.zeros((10, lmax + 1))
    cl_tebp_arr[:, 2:] = _cl_tebp_arr

    return cl_tebp_arr


def run_taylens(self):
    """Returns CMB (T, Q, U) maps as a function of observing frequency, nu.

    This code is extracted from the taylens code (reference).

    :return: function -- CMB maps.
    """
    synlmax = 8 * self.nside  # this used to be user-defined.
    lmax_cl = self.cmb_spectra[0][-1]  # Get the maximum l from the input spectra
    ell = np.arange(int(lmax_cl + 1))
    synlmax = min(synlmax, ell[-1])

    # Convert the input spectra to the format used by taylens
    cl_tebp_arr = camb2tebp(self.cmb_spectra, self.delensing_ells)

    # Coordinates of healpix pixel centers
    ipos = np.array(hp.pix2ang(self.nside, np.arange(hp.nside2npix(self.nside))))

    # Simulate a CMB and lensing field
    cmb, aphi = simulate_tebp_correlated(
        cl_tebp_arr, self.nside, synlmax, self.cmb_seed
    )

    if cmb.ndim == 1:
        cmb = np.reshape(cmb, [1, cmb.size])

    # Compute the offset positions
    phi, phi_dtheta, phi_dphi = hp.alm2map_der1(aphi, self.nside, lmax=synlmax)

    del aphi

    opos, rot = offset_pos(
        ipos, phi_dtheta, phi_dphi, pol=True, geodesic=False
    )  # geodesic used to be used defined.
    del phi, phi_dtheta, phi_dphi

    # Interpolate maps one at a time
    maps = []
    for comp in cmb:
        for m in taylor_interpol_iter(
            comp, opos, 3, verbose=False, lmax=None
        ):  # lmax here needs to be fixed. order of taylor expansion is fixed to 3.
            pass
        maps.append(m)
    del opos, cmb
    return np.array(apply_rotation(maps, rot))
